RNN 和多层感知机的区别是，多了一个 $W_{hh}$ 存储时序信息，“就这么点区别”

## 1.一个只有单隐藏层的多层感知机
### （1） 具有 h 个单元的隐藏层
激活函数： $\phi$,
一个小批量样本： $X \in \mathbb{R}^{n \times d}$  ,其中批量大小为 $n$,输入维度为 $d$
隐藏层权重矩阵： $W_{xh} \in \mathbb{R}^{d \times h}$
隐藏层 bias 参数： $b_h \in \mathbb{R}^{1 \times h}$  ,$1 \times h$ 的向量通过广播机制与 $n \times h$ 的矩阵求和
隐藏层输出： $H \in \mathbb{R}^{n \times h}$ ,其中隐藏层单元数为 $h$

隐藏层输出计算为：
$$H = \phi(XW_{xh}+b_h)$$

### (2) 具有 q 个单元的输出层
权重矩阵 $W_{hq} \in \mathbb{R}^{h \times q} $
输出层 bias 参数： $b_q \in \mathbb{R}^{1 \times q}$
输出变量 $O \in \mathbb{R}^{n \times q}$

输出层计算为：
$$ O = \phi(HW_{hq}+b_q)$$

## 2.添加隐状态，成为RNN
时间步$t$处的小批量样本：$X_t \in \mathbb{R}^{n \times d}$ ,其中 $n$ 对应 `batch_size`,$d$ 对应`num_steps`
时间步$t$的隐藏变量：$H_t \in \mathbb{R}^{n \times h}$,前一个时间步的隐藏变量为$H_{t-1}$
$H_{t-1} \rightarrow H_t$ 是如何更新的呢？引入一个新的权重参数 $W_{hh} \in \mathbb{R}^{h \times h}$

当前 $H_t$ 取决于 前一个隐藏变量 $H_{t-1}$ 和本次的输入$X_t$:
$$ H_t = \phi(X_t W_{xh} + \textcolor{red}{H_{t-1} W_{hh}} + b_h)$$

输出层类似于多层感知机：
$$ O_\textcolor{red}{t} = \phi(H_\textcolor{red}{t}W_{hq}+b_q)$$

RNN的参数包括
- 隐藏层：权重 $W_{xh} \in \mathbb{R}^{d \times h}$，$W_{hh} \in \mathbb{R}^{h \times h}$ 和偏置$b_h \in \mathbb{R}^{1 \times h}$
- 输出层：输出权重 $W_{hq} \in \mathbb{R}^{h \times q}$(h:隐藏层单元数;q:输出层单元数)、偏置$b_q \in \mathbb{R}^{1 \times q}$
※ 即使在不同的时间步，循环神经网络也总是使用这些模型参数。 因此，循环神经网络的参数开销不会随着时间步的增加而增加。

## 一、隐藏矩阵的运算基础
举例如下
一个batch 有 3 个数据，维度均为1
隐藏层单元数=4

In [1]:
import torch
from d2l import torch as d2l

X, W_xh = torch.normal(0, 1, (3, 1)), torch.normal(0, 1, (1, 4))
H, W_hh = torch.normal(0, 1, (3, 4)), torch.normal(0, 1, (4, 4))
X,W_xh,H,W_hh

(tensor([[-0.1290],
         [-1.2654],
         [-0.2820]]),
 tensor([[-0.7099, -2.0811,  1.9391, -0.1431]]),
 tensor([[-5.2354e-02,  6.4240e-01,  4.7464e-01, -8.5403e-01],
         [-1.3654e-01,  1.0121e+00,  4.4602e-01, -3.2529e-01],
         [ 2.9313e-01,  6.6841e-02, -2.3511e+00, -2.0708e-03]]),
 tensor([[ 1.3651, -0.5755,  0.9163,  1.0468],
         [ 1.1635,  0.2138, -0.2524, -0.2797],
         [ 0.8933,  0.8268, -0.7628,  0.6019],
         [-1.3133, -0.4914,  0.2944, -0.8251]]))

隐状态 $X_t  W_{xh} + H_{t-1} W_{hh}$ 的底层计算:
(3,1)\*(1,4)->(3,4)
(3,4)\*(4,4)->(3,4)
(3,4)+(3,4)->(3,4)

In [2]:
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 2.3131,  1.2479, -1.0737,  0.7743],
        [ 2.7151,  3.4570, -3.2702,  0.2919],
        [-1.4193, -1.5105,  1.4976, -1.0850]])

※ 上述计算$$X_t  W_{xh} + H_{t-1} W_{hh}$$ 等于（$X_t$ 和 $H_{t-1}$ 的拼接）与（$W_{xh}$ 和 $W_{hh}$ 的拼接）的矩阵乘法

In [3]:
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 2.3131,  1.2479, -1.0737,  0.7743],
        [ 2.7151,  3.4570, -3.2702,  0.2919],
        [-1.4193, -1.5105,  1.4976, -1.0850]])

## 总结
- RNN 的目标是根据过去的和当前的词元预测下一个词元
- 循环神经网络的输出取决于当下输入和前一时间的隐变量
- 应用到语言模型中时，循环神经网络根据当前词预测下一次时刻词
- 通常使用困惑度来衡量语言模型的好坏